In [4]:
import math
import numpy as np


In [5]:
def set_parameters(otype, M):
    ''' Sets parameters depending on valuation case.

    Parameters
    ==========
    otype : int
        option type
        1 = American put option
        2 = Short Condor Spread
    '''
    if otype == 1:
        # Parameters -- American Put Option
        S0 = 36.  # initial stock level
        T = 1.0  # time-to-maturity
        r = 0.06  # short rate
        sigma = 0.2  # volatility

    elif otype == 2:
        # Parameters -- Short Condor Spread
        S0 = 100.  # initial stock level
        T = 1.0  # time-to-maturity
        r = 0.05  # short rate
        sigma = 0.5  # volatility

    else:
        raise ValueError('Option type not known.')

    # Numerical Parameters
    dt = T / M  # time interval
    df = math.exp(-r * dt)  # discount factor
    u = math.exp(sigma * math.sqrt(dt))  # up-movement
    d = 1 / u  # down-movement
    q = (math.exp(r * dt) - d) / (u - d)  # martingale probability

    return S0, T, r, sigma, M, dt, df, u, d, q


def inner_value(S, otype):
    ''' Inner value functions for American put option and short condor spread
    option with American exercise.

    Parameters
    ==========
    otype : int
        option type
        1 = American put option
        2 = Short Condor Spread
    '''
    if otype == 1:
        return np.maximum(40. - S, 0)
    elif otype == 2:
        return np.minimum(40., np.maximum(90. - S, 0) + 
                          np.maximum(S - 110., 0))
    else:
        raise ValueError('Option type not known.')


def CRR_option_valuation(otype, M=500):
    S0, T, r, sigma, M, dt, df, u, d, q = set_parameters(otype, M)
    # Array Generation for Stock Prices
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md

    # Valuation by Backwards Induction
    h = inner_value(S, otype)  # innver value matrix
    V = inner_value(S, otype)  # value matrix
    C = np.zeros((M + 1, M + 1), dtype=np.float)  # continuation values
    ex = np.zeros((M + 1, M + 1), dtype=np.float)  # exercise matrix

    z = 0
    for i in range(M - 1, -1, -1):
        C[0:M - z, i] = (q * V[0:M - z, i + 1] +
                         (1 - q) * V[1:M - z + 1, i + 1]) * df
        V[0:M - z, i] = np.where(h[0:M - z, i] > C[0:M - z, i],
                                 h[0:M - z, i], C[0:M - z, i])
        ex[0:M - z, i] = np.where(h[0:M - z, i] > C[0:M - z, i], 1, 0)
        z += 1
    return V[0, 0]


In [6]:
%time 

CRR_option_valuation(1, 500)



CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs


<ipython-input-5-31aa4241f0d0>:71: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  C = np.zeros((M + 1, M + 1), dtype=np.float)  # continuation values
<ipython-input-5-31aa4241f0d0>:72: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ex = np.zeros((M + 1, M + 1), dtype=np.float)  # exercise matrix


4.48637477750599

In [11]:
# Valuation of American Option
# With Least Square Monte Carlo.

import time
np.random.seed(150000)

# Model Parameters
S0 = 36.  # initial stock level
K = 40.  # strike price
T = 1.0  # time-to-maturity
r = 0.06  # short rate
sigma = 0.2  # volatility

In [17]:
# Stock Price Paths
S = S0 * np.exp(np.cumsum((r - 0.5 * sigma ** 2) * dt +
                          sigma * math.sqrt(dt) *
                          np.random.standard_normal((M + 1, I)), axis=0))


In [12]:
start_time=time.time()
# Simulation Parameters
I = 25000
M = 50
dt = T / M
df = math.exp(-r * dt)

# Stock Price Paths
S = S0 * np.exp(np.cumsum((r - 0.5 * sigma ** 2) * dt +
                          sigma * math.sqrt(dt) *
                          np.random.standard_normal((M + 1, I)), axis=0))
S[0] = S0

# Inner Values
h = np.maximum(K - S, 0)

# Present Value Vector (Initialization)
V = h[-1]

# American Option Valuation by Backwards Induction
for t in range(M - 1, 0, -1):
    rg = np.polyfit(S[t], V * df, 5)
    C = np.polyval(rg, S[t])  # continuation values
    V = np.where(h[t] > C, h[t], V * df)
    # exercise decision
V0 = df * np.sum(V) / I  # LSM estimator

end_time=time.time()
print('Time for valuations: ',end_time-start_time)
print("American put option value %5.3f" % V0)



Time for valuations:  0.2718698978424072
American put option value 4.475


In [ ]:
# Valuation of American Options
# with Least-Squares Monte Carlo
# Primal and Dual Algorithm
# Case 1: American Put Option (APO)
# Case 2: Short Condor Spread (SCS)
# 07_amo/LSM_primal_dual_valuation.py
#

In [13]:
import math
import numpy as np
import pandas as pd
from time import time
from datetime import datetime
import itertools as it
import warnings
warnings.simplefilter('ignore')

t0 = time()
np.random.seed(150000)  # seed for Python RNG

In [14]:
# Simulation Parameters
runs = 5
write = True
otype = [1, 2]  # option type
M = [10, 20]  # time steps
I1 = np.array([4, 6]) * 4096  # replications for regression
I2 = np.array([1, 2]) * 1024  # replications for valuation
J = [50, 75]  # replications for nested MCS
reg = [5, 9]  # no of basis functions
AP = [False, True]  # antithetic paths
MM = [False, True]  # moment matching of RN
ITM = [True, False]  # ITM paths for regression

results = pd.DataFrame()

In [15]:
#
# Function Definitions
#


def generate_random_numbers(I):
    ''' Function to generate I pseudo-random numbers. '''
    if AP:
            ran = np.random.standard_normal(I / 2)
            ran = np.concatenate((ran, -ran))
    else:
            ran = np.random.standard_normal(I)
    if MM:
        ran = ran - np.mean(ran)
        ran = ran / np.std(ran)
    return ran


def generate_paths(I):
    ''' Function to generate I stock price paths. '''
    S = np.zeros((M + 1, I), dtype=np.float)  # stock matrix
    S[0] = S0  # initial values
    for t in range(1, M + 1, 1):  # stock price paths
        ran = generate_random_numbers(I)
        S[t] = S[t - 1] * np.exp((r - sigma ** 2 / 2) * dt +
                                 sigma * ran * math.sqrt(dt))
    return S


def inner_values(S):
    ''' Innver value functions for American put and Short Condor Spread. '''
    if otype == 1:
        return np.maximum(40. - S, 0)
    else:
        return np.minimum(40., np.maximum(90. - S, 0) +
                          np.maximum(S - 110., 0))


def nested_monte_carlo(St, J):
    ''' Function for nested Monte Carlo simulation.

    Parameters
    ==========
    St : float
        start value for S
    J : int
        number of paths to simulate

    Returns
    =======
    paths : array
        simulated nested paths
    '''
    ran = generate_random_numbers(J)
    paths = St * np.exp((r - sigma ** 2 / 2) * dt +
                        sigma * ran * math.sqrt(dt))
    return paths

In [16]:
#
# Valuation
#
para = it.product(otype, M, I1, I2, J, reg, AP, MM, ITM)
count = 0
for pa in para:
    otype, M, I1, I2, J, reg, AP, MM, ITM = pa
    # General Parameters and Option Values
    if otype == 1:
        # Parameters -- American Put Option
        S0 = 36.  # initial stock level
        T = 1.0  # time-to-maturity
        r = 0.06  # short rate
        sigma = 0.2  # volatility
        V0_true = 4.48637  # American Put Option (500 steps bin. model)
    else:
        # Parameters -- Short Condor Spread
        S0 = 100.  # initial stock level
        T = 1.0  # time-to-maturity
        r = 0.05  # short rate
        sigma = 0.5  # volatility
        V0_true = 26.97705  # Short Condor Spread (500 steps bin. model)
    dt = T / M  # length of time interval
    df = math.exp(-r * dt)  # discount factor per time interval
    for j in range(runs):
        count += 1
        # regression estimation
        S = generate_paths(I1)  # generate stock price paths
        h = inner_values(S)  # inner values
        V = inner_values(S)  # value matrix
        # regression parameter matrix
        rg = np.zeros((M + 1, reg + 1), dtype=np.float)

        itm = np.greater(h, 0)  # ITM paths
        for t in range(M - 1, 0, -1):
            if ITM:
                S_itm = np.compress(itm[t] == 1, S[t])
                V_itm = np.compress(itm[t] == 1, V[t + 1])
                if len(V_itm) == 0:
                    rg[t] = 0.0
                else:
                    rg[t] = np.polyfit(S_itm, V_itm * df, reg)
            else:
                # regression at time t
                rg[t] = np.polyfit(S[t], V[t + 1] * df, reg)
            C = np.polyval(rg[t], S[t])  # continuation values
            # exercise decision
            V[t] = np.where(h[t] > C, h[t], V[t + 1] * df)

        # Simulation
        Q = np.zeros((M + 1, I2), dtype=np.float)  # martingale matrix
        U = np.zeros((M + 1, I2), dtype=np.float)  # upper bound matrix
        S = generate_paths(I2)  # generate stock price paths
        h = inner_values(S)  # inner values
        V = inner_values(S)  # value matrix

        # Primal Valuation
        for t in range(M - 1, 0, -1):
            C = np.polyval(rg[t], S[t])  # continuation values
            # exercise decision
            V[t] = np.where(h[t] > C, h[t], V[t + 1] * df)
        V0 = df * np.sum(V[1]) / I2  # LSM estimator

        # Dual Valuation
        for t in range(1, M + 1):
            for i in range(I2):
                Vt = max(h[t, i], np.polyval(rg[t], S[t, i]))
                # estimated value V(t,i)
                St = nested_monte_carlo(S[t - 1, i], J)  # nested MCS
                Ct = np.polyval(rg[t], St)  # cv from nested MCS
                ht = inner_values(St)  # iv from nested MCS
                # average of V(t,i,j)
                VtJ = np.sum(np.where(ht > Ct, ht, Ct)) / len(St)
                Q[t, i] = Q[t - 1, i] / df + (Vt - VtJ)  # "optimal" martingale
                # high estimator values
                U[t, i] = max(U[t - 1, i] / df, h[t, i] - Q[t, i])
                if t == M:
                    U[t, i] = np.maximum(U[t - 1, i] / df,
                                         np.mean(ht) - Q[t, i])
        U0 = np.sum(U[M]) / I2 * df ** M  # DUAL estimator
        AV = (V0 + U0) / 2  # average of LSM and DUAL estimator

        # output
        print("%4d | %4.1f | %48s " % (count, (time() - t0) / 60, pa),
              "| %6.3f | %6.3f | %6.3f" % (V0, U0, AV))
        # results storage
        results = results.append(pd.DataFrame({'otype': otype, 'runs': runs,
            'M': M, 'I1': I1, 'I2': I2, 'J': J, 'reg': reg, 'AP': AP,
            'MM': MM, 'ITM': ITM, 'LSM': V0, 'LSM_se': (V0 - V0_true) ** 2,
            'DUAL': U0, 'DUAL_se': (U0 - V0_true) ** 2, 'AV': AV,
            'AV_se': (AV - V0_true) ** 2}, index=[0,]), ignore_index=True)

t1 = time()
print("Total time in min %s" % ((t1 - t0) / 60))

if write:
    h5 = pd.HDFStore('results_%s_%s.h5' % (datetime.now().date(),
                     str(datetime.now().time())[:8]), 'w')
    h5['results'] = results
    h5.close()


   1 |  0.1 |  (1, 10, 16384, 1024, 50, 5, False, False, True)  |  4.517 |  7.430 |  5.973
   2 |  0.1 |  (1, 10, 16384, 1024, 50, 5, False, False, True)  |  4.284 |  9.051 |  6.668
   3 |  0.1 |  (1, 10, 16384, 1024, 50, 5, False, False, True)  |  4.208 |  7.120 |  5.664
   4 |  0.1 |  (1, 10, 16384, 1024, 50, 5, False, False, True)  |  4.387 |  8.507 |  6.447
   5 |  0.1 |  (1, 10, 16384, 1024, 50, 5, False, False, True)  |  4.167 |  9.298 |  6.732
   6 |  0.1 | (1, 10, 16384, 1024, 50, 5, False, False, False)  |  4.505 |  4.910 |  4.708
   7 |  0.1 | (1, 10, 16384, 1024, 50, 5, False, False, False)  |  4.352 |  4.868 |  4.610
   8 |  0.1 | (1, 10, 16384, 1024, 50, 5, False, False, False)  |  4.378 |  4.905 |  4.642
   9 |  0.2 | (1, 10, 16384, 1024, 50, 5, False, False, False)  |  4.488 |  4.897 |  4.693
  10 |  0.2 | (1, 10, 16384, 1024, 50, 5, False, False, False)  |  4.493 |  4.901 |  4.697
  11 |  0.2 |   (1, 10, 16384, 1024, 50, 5, False, True, True)  |  4.370 |  6.876 |  5.623

TypeError: 'numpy.float64' object cannot be interpreted as an integer